# Fake Analysis

This notebook provides a short example of analyzing the fakes from the DEEP data using `analyze_fakes.py`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from kbmod.analysis.analyze_fakes import FakeInfo, load_fake_info_from_ecsv
from kbmod.analysis.plotting import plot_image
from kbmod.work_unit import WorkUnit

import logging

logging.basicConfig(level=logging.INFO)

Load the test WorkUnit and extract some metadata (t0).

In [ ]:
wu_file = "/epyc/projects/kbmod/data/20210908_B1h_047_test_data/20210908_B1h_047.wu"
wu = WorkUnit.from_fits(wu_file)
times = wu.get_all_obstimes()
t0 = times[0]

print(f"Loaded {len(wu)} images starting at time {t0}")

We can compute basic statistics for the WorkUnit.

In [ ]:
wu.print_stats()

Load the fakes data from the ecsv file. For each object (unique orbitid), build a trajectory with from those observations.  We use the `load_fake_info_from_ecsv()` helper function.

In [ ]:
fakes_file = "/epyc/projects/kbmod/data/20210908_B1h_047_test_data/20210908_B1h_047_inserted_fakes.ecsv"
fakes_list = load_fake_info_from_ecsv(fakes_file)

We now have a list of `FakeInfo` objects with the minimal information (the inserted fake's time, RA, and dec).  To get meaningful information, we need to join against what we know about the images. We use the images' WCS to compute the fake object's pixel positions at each time. Then we use those pixel positions to extract stamps and fit a linear trajectory.

Currently we join with each `FakeInfo` object separately (we could add a helper function here if needed).

In [ ]:
for fake in fakes_list:
    fake.join_with_workunit(wu, 10)
    print(f"{fake.name}:\n  Fit:{fake.trj}\n  MSE={fake.compute_fit_mse()}")

# Examining Stamps

We can plot the stamps at the raw (x, y) that we computed from the (RA, dec) position and the image WCS. We can also look at the positions predicted by the fitted, linear trajectory.  Below we look at the stamps at the first 4 time steps.

In [ ]:
fakes_list[0].compare_stamps([0, 1, 2, 3])